In [ ]:
!pip3 install ortools

In [ ]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP')
#why GLOP doesnt work?


import pandas as pd
from datetime import datetime


def calculate_hourly_rate_chef(day, public_holiday):
  if public_holiday:
    return 17
  elif day == 'Saturday' or day == 'Sunday':
      return 16
  else:
      return 15

def calculate_hourly_rate(day, public_holiday):
  if public_holiday:
    return 16
  elif day == 'Saturday' or day == 'Sunday':
    return 15
  else:
    return 14

def calculate_hourly_rate_part(day, public_holiday):
  if public_holiday:
    return 15
  elif day == 'Saturday' or day == 'Sunday':
    return 14
  else:
    return 13


def Requirements_chef(C_Buffet, I_Reserve):
  Req = []
  for i in range(7):
    Req.append([[4, 4, 1], [1, 3, 1],  [0, 0, 0]])

  for i in range(7):
    if C_Buffet[i]:
      Req[i][1][0] += 1
      Req[i][1][1] += 1
    if I_Reserve[i]:
      Req[i][2][0] += 1
      Req[i][2][1] += 1
      Req[i][2][2] += 1
  return Req

def Requirements(C_Buffet, I_Buffet, I_Reserve):
  Req = []
  for i in range(7):
    Req.append([[4, 4, 1], [1, 3, 1],  [0, 0, 0]])

  for i in range(7):
    if C_Buffet[i]:
      Req[i][1][0] += 1
      Req[i][1][1] += 1
    if I_Reserve[i]:
      Req[i][2][0] += 1
      Req[i][2][1] += 2
      Req[i][2][2] += 1
  return Req


def smart_Schedule(input):

    Chinese = input['Chinese_Buffet_Busy'].tolist()
    Indian = input['Indian_Buffet_Busy'].tolist()
    Indian_R = input['India_Reservation'].tolist()

    req = Requirements(Chinese, Indian, Indian_R)
    req_chef = Requirements_chef(Chinese, Indian_R)

    day_status = []
    for idx, row in input.iterrows():
      dat = row['Date']
      date = datetime.strptime(dat, '%d/%m/%Y')
      day = row['Day']
      public_holiday = row['Public Holiday']
      if public_holiday:
        day_status.append(2)
      elif day == 'Saturday' or day == 'Sunday':
        day_status.append(1)
      else:
        day_status.append(0)


    def kitchen_Schedule():
        solver = pywraplp.Solver.CreateSolver('SCIP')
        variables = [solver.IntVar(0, 1, f'x{i}') for i in range(1, 351)]

        # Minimum hours
        for i in range(5):
            constraint = solver.Constraint(40, solver.infinity())
            for j in range(i*70, (i+1)*70):
                constraint.SetCoefficient(variables[j], 1)

        # Schedule meets requirements for the week
        kitchen_req = [[], [], [], [], [], [], []]
        for i in range(7):
            for j in range(3):
                if j == 0: kitchen_req[i].extend([req_chef[i][j][0] for k in range(7)])
                elif j == 1: kitchen_req[i].extend([req_chef[i][j][0] for k in range(3)])
                else:
                    for k in range(2): kitchen_req[i][-(k+1)] += req_chef[i][j][0]
        for i in range(70):
            constraint = solver.Constraint(kitchen_req[i//10][i%10], solver.infinity())
            for j in range(5):
                constraint.SetCoefficient(variables[i+(j*70)], 1)

        # At least one day off
        # Idea: A1 off on least bziest day, A2 off on 2nd least bziest day, ..., A5 off on 5th least bziest day
        Rank = [kitchen_req[i][9] for i in range(7)]
        info = [i[0] for i in sorted(enumerate(Rank), key=lambda x:x[1])]
        lst = []
        for i in range(7):
            for j in range(7):
                if i == info[j]: lst.append(j)

        for i in range(5):
            constraint = solver.Constraint(0, 0)
            key = (i*70) + (lst[i]*10)
            for j in range(key, key+10):
                constraint.SetCoefficient(variables[j], 1)

        # Cost function
        objective = solver.Objective()
        cnt = 0
        for i in range(5):
            for j in range(7):
                for k in range(10):
                    tmp = 15 + day_status[j]
                    objective.SetCoefficient(variables[cnt], tmp)
                    cnt += 1
        objective.SetMinimization

        # Solve
        status = solver.Solve()
        lst = []
        if status == pywraplp.Solver.OPTIMAL:
            for i, var in enumerate(variables):
                lst.append(var.solution_value())
            roster = [[[], [], [], [], [], [], [], [], [], []] for i in range(7)]
            names = ["A1", "A2", "A3", "A4", "A5"]
            cnt = 0
            for workers in range(5):
                for days in range(7):
                    for hour_shift in range(10):
                        if lst[cnt] == 1: roster[days][hour_shift].append(names[workers])
                        cnt +=1

        return roster

    def SP_Schedule():
        solver = pywraplp.Solver.CreateSolver('SCIP')
        variables = [solver.IntVar(0, 1, f'x{i}') for i in range(1, 561)]

        # Minimum hours for Full-Time service and Part-Time
        for i in range(5):
            tmp = 33
            if i >= 3: tmp =40
            constraint = solver.Constraint(tmp, solver.infinity())
            for j in range(i*70, (i+1)*70):
                constraint.SetCoefficient(variables[j], 1)

        # Schedule meets requirements for the week
        service_req = [[], [], [], [], [], [], []]
        for i in range(7):
            for j in range(3):
                # Lunch hours
                if j == 0: service_req[i].extend([req[i][j][1] for k in range(7)])
                # Chinese Buffet
                elif j == 1: service_req[i].extend([req[i][j][1] for k in range(3)])
                # Indian Buffet
                else:
                    for k in range(2): service_req[i][-(k+1)] += req[i][j][1]


        for i in range(70):
            constraint = solver.Constraint(service_req[i//10][i%10], solver.infinity())
            for j in range(8):
                constraint.SetCoefficient(variables[i+(j*70)], 1)

        # At least one day off
        # Idea: D1, D2 off on least bziest day, D3 off on 2nd least bziest day, ..., (D1 D2 D3 B1 B2 B3 B4 B5)
        Rank = [service_req[i][9] for i in range(7)]
        info = [i[0] for i in sorted(enumerate(Rank), key=lambda x:x[1])]
        lst = []
        for i in range(7):
            for j in range(7):
                if i == info[j]: lst.append(j)

        ## 2 ppl go on off
        constraint = solver.Constraint(0, 0)
        for i in range(10):
            tmp1 = lst[0]*10
            tmp2 = 70 + tmp1
            constraint.SetCoefficient(variables[tmp1+i], 1)
            constraint.SetCoefficient(variables[tmp2+i], 1)
        ## 1 ppl go on off
        for i in range(6):
            constraint = solver.Constraint(0, 0)
            key = ((i+2)*70) + (lst[i+1]*10)
            for j in range(key, key+10):
                constraint.SetCoefficient(variables[j], 1)


        # Cost function
        objective = solver.Objective()
        cnt = 0
        for i in range(8):
            for j in range(7):
                for k in range(10):
                    tmp = 13 + day_status[j]
                    if i >= 3: tmp += 1
                    objective.SetCoefficient(variables[cnt], tmp)
                    cnt += 1
        objective.SetMinimization

        # Solve
        status = solver.Solve()
        lst = []
        if status == pywraplp.Solver.OPTIMAL:
            for i, var in enumerate(variables):
                lst.append(var.solution_value())
            roster = [[[], [], [], [], [], [], [], [], [], []] for i in range(7)]
            names = ["D1", "D2", "D3", "B1", "B2", "B3", "B4", "B5"]
            cnt = 0
            for workers in range(8):
                for days in range(7):
                    for hour_shift in range(10):
                        if lst[cnt] == 1: roster[days][hour_shift].append(names[workers])
                        cnt +=1

        return roster

    def dish_Schedule():
        solver = pywraplp.Solver.CreateSolver('SCIP')
        variables = [solver.IntVar(0, 1, f'x{i}') for i in range(1, 141)]

        # Minimum hours
        for i in range(2):
            constraint = solver.Constraint(40, solver.infinity())
            for j in range(i*70, (i+1)*70):
                constraint.SetCoefficient(variables[j], 1)

        # Schedule meets requirements for the week
        dish_req = [[], [], [], [], [], [], []]
        for i in range(7):
            for j in range(3):
                if j == 0: dish_req[i].extend([req[i][j][2] for k in range(7)])
                elif j == 1: dish_req[i].extend([req[i][j][2] for k in range(3)])
                else:
                    for k in range(2): dish_req[i][-(k+1)] += req[i][j][2]
        for i in range(70):
            constraint = solver.Constraint(dish_req[i//10][i%10], solver.infinity())
            for j in range(2):
                constraint.SetCoefficient(variables[i+(j*70)], 1)

        # At least one day off
        # Idea: Need 2 least buziest day that is not 2 dishwashers if only everyday except one is 2 dish, then prioritise C1
        Rank = [dish_req[i][9] for i in range(7)]
        info = [i[0] for i in sorted(enumerate(Rank), key=lambda x:x[1])]
        lst = []
        for i in range(7):
            for j in range(7):
                if i == info[j]: lst.append(j)

        if Rank[lst[0]] < 2:
          constraint = solver.Constraint(0, 0)
          tmp1 = lst[0]*10
          for i in range(10):
                constraint.SetCoefficient(variables[tmp1+i], 1)
        if Rank[lst[1]] < 2:
          constraint = solver.Constraint(0, 0)
          tmp2 = lst[1]*10
          for i in range(10):
              constraint.SetCoefficient(variables[tmp2+i], 1)

        # Cost function
        objective = solver.Objective()
        cnt = 0
        for i in range(2):
            for j in range(7):
                for k in range(10):
                    tmp = 14 + day_status[j]
                    objective.SetCoefficient(variables[cnt], tmp)
                    cnt += 1
        objective.SetMinimization

        # Solve
        status = solver.Solve()
        lst = []
        if status == pywraplp.Solver.OPTIMAL:
            for i, var in enumerate(variables):
                lst.append(var.solution_value())
            roster = [[[], [], [], [], [], [], [], [], [], []] for i in range(7)]
            names = ["C1", "C2"]
            cnt = 0
            for workers in range(2):
                for days in range(7):
                    for hour_shift in range(10):
                        if lst[cnt] == 1: roster[days][hour_shift].append(names[workers])
                        cnt +=1

        return roster

    kitchen = kitchen_Schedule()
    serve = SP_Schedule()
    dish = dish_Schedule()

    full_Schedule = kitchen
    for i in range(7):
        for j in range(10):
            full_Schedule[i][j].extend(serve[i][j])
            full_Schedule[i][j].extend(dish[i][j])
    return full_Schedule


def transform():
  chefs = ['A1', 'A2', 'A3', 'A4', 'A5']
  service = ['B1', 'B2', 'B3', 'B4', 'B5']
  dishwashers = ['C1', 'C2']
  parttimers = ['D1', 'D2', 'D3']
  schedule_data = []
  for chunk in pd.read_csv('predictions.csv', chunksize=7):
    chunk = chunk.reset_index(drop=True)
    final_schedule = smart_Schedule(chunk)

    for idx, row in chunk.iterrows():
      dat = row['Date']
      date = datetime.strptime(dat, '%d/%m/%Y')
      day = row['Day']
      public_holiday = row['Public Holiday']
      hourly_rate_chef = calculate_hourly_rate_chef(day, public_holiday)
      hourly_rate = calculate_hourly_rate(day, public_holiday)
      hourly_rate_part = calculate_hourly_rate_part(day, public_holiday)

      for employee_id in final_schedule[idx][0]:
        if employee_id in chefs:
          schedule_data.append([date, day, public_holiday, employee_id, '10am-4.30pm', 'chef', 6.5, hourly_rate_chef, 'full-time'])
        elif employee_id in service:
          schedule_data.append([date, day, public_holiday, employee_id, '10am-4.30pm', 'service', 6.5, hourly_rate, 'full-time'])
        elif employee_id in parttimers:
          schedule_data.append([date, day, public_holiday, employee_id, '10am-4.30pm', 'service', 6.5, hourly_rate_part, 'part-time'])
        else:
          schedule_data.append([date, day, public_holiday, employee_id, '10am-4.30pm', 'dishwasher', 6.5, hourly_rate, 'full-time'])


      for employee_id in final_schedule[idx][7]:
        if employee_id in chefs:
          schedule_data.append([date, day, public_holiday, employee_id, '7pm-10pm', 'chef', 3, hourly_rate_chef, 'full-time'])
        elif employee_id in service:
          schedule_data.append([date, day, public_holiday, employee_id, '7pm-10pm', 'service', 3, hourly_rate, 'full-time'])
        elif employee_id in parttimers:
          schedule_data.append([date, day, public_holiday, employee_id, '7pm-10pm', 'service', 3, hourly_rate_part, 'part-time'])
        else:
          schedule_data.append([date, day, public_holiday, employee_id, '7pm-10pm', 'dishwasher', 3, hourly_rate, 'full-time'])


      for employee_id in final_schedule[idx][8]:
        if schedule_data:
          for rw in schedule_data:
            if employee_id in rw and date in rw and rw[4] == '7pm-10pm':
              break
          else:
            if employee_id in chefs:
              schedule_data.append([date, day, public_holiday, employee_id, '8pm-10pm', 'chef', 2, hourly_rate_chef, 'full-time'])
            elif employee_id in service:
              schedule_data.append([date, day, public_holiday, employee_id, '8pm-10pm', 'service', 2, hourly_rate, 'full-time'])
            elif employee_id in parttimers:
              schedule_data.append([date, day, public_holiday, employee_id, '8pm-10pm', 'service', 2, hourly_rate_part, 'part-time'])
            else:
              schedule_data.append([date, day, public_holiday, employee_id, '8pm-10pm', 'dishwasher', 2, hourly_rate, 'full-time'])

  final_sched = pd.DataFrame(schedule_data, columns=['Date', 'Day', 'Public Holiday', 'Employee_ID', 'Shift', 'Role', 'Hours_worked', 'Hourly_rate', 'Job_status'])
  final_sched['Date'] = pd.to_datetime(final_sched['Date'], errors='coerce')
  final_sched = final_sched.sort_values(by=['Date', 'Shift', 'Employee_ID'])
  final_sched.dropna(subset=['Date'], inplace=True)
  final_sched.to_csv('final_schedule.csv', index=False)

transform()